In [1]:
%%HTML
<style>

/* To make it easier to read the pre and code components */

.rendered_html pre {
    background-color : #fffddd !important;
}

.rendered_html pre code {
    background-color: transparent !important;
}
</style>

# Day 7
## Part 1

You land at the regional airport in time for your next flight. In fact, it looks like you'll even have time to grab some food: all flights are currently delayed due to issues in luggage processing.

Due to recent aviation regulations, many rules (your puzzle input) are being enforced about bags and their contents; bags must be color-coded and must contain specific quantities of other color-coded bags. Apparently, nobody responsible for these regulations considered how long they would take to enforce!

For example, consider the following rules:

```
* light red bags contain 1 bright white bag, 2 muted yellow bags.
* dark orange bags contain 3 bright white bags, 4 muted yellow bags.
* bright white bags contain 1 shiny gold bag.
* muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
* shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
* dark olive bags contain 3 faded blue bags, 4 dotted black bags.
* vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
* faded blue bags contain no other bags.
* dotted black bags contain no other bags.
```

These rules specify the required contents for 9 bag types. In this example, every faded blue bag is empty, every vibrant plum bag contains 11 bags (5 faded blue and 6 dotted black), and so on.

You have a shiny gold bag. If you wanted to carry it in at least one other bag, how many different bag colors would be valid for the outermost bag? (In other words: how many colors can, eventually, contain at least one shiny gold bag?)

In the above rules, the following options would be available to you:

* A bright white bag, which can hold your shiny gold bag directly.
* A muted yellow bag, which can hold your shiny gold bag directly, plus some other bags.
* A dark orange bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny gold bag.
* A light red bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny gold bag.

So, in this example, the number of bag colors that can eventually contain at least one shiny gold bag is 4.

How many bag colors can eventually contain at least one shiny gold bag? (The list of rules is quite long; make sure you get all of it.)

### Include essential libraries

In [2]:
# To best solve this problem, you can try two ways:
# Adjancency Matrix
# Graph Data Structure

# We'll try solving with both

# Import regex and numpy
import re
import numpy as np

# PrettyPrint
from pprint import pprint

# Time to measure performance of each method
import time

### Generate the necessary data structure

In [3]:
# Create a function that cleans the input and outputs a matrix
def matrix_and_graph_generator(data_input):
    
    # Regex
    re_key = r'(\S.+) bags contain (.*)?'
    re_val = r'(\d+) ([ A-z]+) bag?'
    
    # Capture the size of the problem
    size = 0

    # A list of keys (for indexing)
    keys = list()

    # A hash table
    graph = {}

    # Iterate
    for line in data_input:

        # Iterate
        size += 1

        # Collect the key via regex
        key = re.match(re_key, line)[1]
        keys.append(key)

        # Collect the values
        values = [(v[1],int(v[0])) for v in re.findall(re_val,line)]

        # Build the hash
        graph[key] = dict([*values])

    # Create a template for the adjacency matrix
    matrix = np.zeros(shape=(size,size), dtype=int)
    

    # Create a dictionary of key name and index/position
    keys = {v:k for k,v in enumerate(keys)}

    # Generate the matrix
    for key in graph:

        # If this is a terminal node, do not continue (i.e. there's nothing 'from' this node 'to' another node)
        if graph[key]:

            # Collect the i (row) value to set the FROM node
            i = keys[key]

            # Let's now update the column value to set the TO node
            for neighbor in graph[key]:
                j = keys[neighbor]
                matrix[i][j] = graph[key][neighbor]
                
    return [matrix, graph, keys]

### Build out different data samples

In [4]:
data_1 = """light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.""".split("\n")

matrix_1, graph_1, keys_1 = matrix_and_graph_generator(data_1)

data_2 = """shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags.""".split("\n")

matrix_2, graph_2, keys_2 = matrix_and_graph_generator(data_2)

# Access the file
with open('inputs/day_7.txt') as reader:
    matrix_3, graph_3, keys_3 = matrix_and_graph_generator(reader)
    
# Test it out
pprint(graph_1)
print(matrix_1)

{'bright white': {'shiny gold': 1},
 'dark olive': {'dotted black': 4, 'faded blue': 3},
 'dark orange': {'bright white': 3, 'muted yellow': 4},
 'dotted black': {},
 'faded blue': {},
 'light red': {'bright white': 1, 'muted yellow': 2},
 'muted yellow': {'faded blue': 9, 'shiny gold': 2},
 'shiny gold': {'dark olive': 1, 'vibrant plum': 2},
 'vibrant plum': {'dotted black': 6, 'faded blue': 5}}
[[0 0 1 2 0 0 0 0 0]
 [0 0 3 4 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 2 0 0 9 0]
 [0 0 0 0 0 1 2 0 0]
 [0 0 0 0 0 0 0 3 4]
 [0 0 0 0 0 0 0 5 6]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]


### Building Find Bag for both data structures

In [5]:
# Matrix Algorithm
def find_bags_matrix(matrix, bag, keys):
    
    # Capture the row names that can host the bag
    results = []
    
    # Identify the TO node (column) and then find where the movement is coming FROM (row)
    columns = [keys[bag]]
    
    while columns:
        
        # Pop the last column
        col = columns.pop()
        
        # Loop through the rows
        for row in range(len(matrix)):
            
            if matrix[row][col] > 0:
                columns.append(row) # Take the row index as the column index (traversing backwards)
                if row not in results and row != keys[bag]:
                    results.append(row)
                    
    return results

# Graph Algorithm
def find_bags_graph(graph, bag):
    
    # Capture the nodes that we're interested in
    results = []
    
    # Create a reverse graph, where the key is the destination, 
    # and children are the nodes to that destination (along with the weight of edge)
    
    reverse_graph = dict()
    
    for k,v in graph.items():
        
        # Setup the initial nodes of our reverse graph
        if k not in reverse_graph:
            reverse_graph[k] = dict()
        
        # Parse through the original graph, and invert it
        for destination, weight in v.items():
            if destination not in reverse_graph:
                reverse_graph[destination] = {
                    k:weight
                }
            else:
                reverse_graph[destination][k] = weight
    
    # pprint(reverse_graph)
    
    # Now that we have our reverse graph, let's traverse!
    path = [bag] # Our starting point
    traversed_path = [] # To record what we've explored already
    
    while path:
        
        # Let's get the nodes we want
        node = path.pop()
        
        # Let's not go through the same node again
        if node not in traversed_path:
            
            # Capture the keys in the node (the parents we're interested in)
            # And establish them as new path
            for parent in list(reverse_graph[node].keys()):
                
                
                if parent not in results:
                    # Record the result!
                    results.append(parent)
                
                # Update the path
                path.append(parent)
            
            # Record it for future use
            traversed_path.append(node)
                
    return results

### Measure performance of each method (Sample Data)

In [6]:
# Let's compare speeds for the sample question

# Matrix
start_matrix = time.time()
answer_matrix = len(find_bags_matrix(matrix_1, "shiny gold", keys_1)) # Answer
end_matrix = time.time()

# Graph
start_graph = time.time()
answer_graph = len(find_bags_graph(graph_1, "shiny gold")) # Answer
end_graph = time.time()

# Result
print("Matrix method answer is {}, it took {:.2}s to process this request".format(answer_matrix, end_matrix - start_matrix))
print("Graph method answer is {}, it took {:.2}s to process this request".format(answer_graph, end_graph - start_graph))

Matrix method answer is 4, it took 9.7e-05s to process this request
Graph method answer is 4, it took 5.4e-05s to process this request


### Measure performance of each method (Puzzle Data)

In [7]:
# Let's compare speeds for the sample question

# Matrix
start_matrix = time.time()
answer_matrix = len(find_bags_matrix(matrix_3, "shiny gold", keys_3)) # Answer
end_matrix = time.time()

# Graph
start_graph = time.time()
answer_graph = len(find_bags_graph(graph_3, "shiny gold")) # Answer
end_graph = time.time()

# Result
print("Matrix method answer is {}, it took {:.2}s to process this request".format(answer_matrix, end_matrix - start_matrix))
print("Graph method answer is {}, it took {:.2}s to process this request".format(answer_graph, end_graph - start_graph))

Matrix method answer is 126, it took 0.047s to process this request
Graph method answer is 126, it took 0.00091s to process this request


## Part 2

It's getting pretty expensive to fly these days - not because of ticket prices, but because of the ridiculous number of bags you need to buy!

Consider again your shiny gold bag and the rules from the above example:

* faded blue bags contain 0 other bags.
* dotted black bags contain 0 other bags.
* vibrant plum bags contain 11 other bags: 5 faded blue bags and 6 dotted black bags.
* dark olive bags contain 7 other bags: 3 faded blue bags and 4 dotted black bags.

So, a single shiny gold bag must contain 1 dark olive bag (and the 7 bags within it) plus 2 vibrant plum bags (and the 11 bags within each of those): 

1 + 1\*7 + 2 + 2\*11 = **32 bags!**

Of course, the actual rules have a small chance of going several levels deeper than this example; be sure to count all of the bags, even if the nesting becomes topologically impractical!

Here's another example:

```
shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags.
```

In this example, a single shiny gold bag must contain 126 other bags.

How many individual bags are required inside your single shiny gold bag?

In [28]:
# Matrix Algorithm
def count_bags_matrix(matrix, bag_row, multiplier = 1):
    """
    Basically, given the matrix, we want to traverse backwards and collect the weights
    inside each cell of the matrix. We'll use multipliers, a variable that changes as we go down
    the rabbit hole using a Depth-first search.
    
    Multiplier = Weight calculatorc compounder
    """
    
    total = 0
    
    # Starter node
    rows = [bag_row] # Basically the bag we're interested in
    
    while rows:
        
        # Pop the last columns
        row = rows.pop()
        
        # Loop through the rows
        for col in range(len(matrix)):
            
            # Find an instance where it exists
            if matrix[row][col] > 0:
                
                # Create a new multiplier instance
                x = multiplier * matrix[row][col] # Collect the value in that cell and multiply with multiplier
                
                total += x # Add to the total
                
                # Let's solve this recursively
                total += count_bags_matrix(matrix,col,x) # Go down the rabbit hole and search deeper (Depth-First search)          
    
    # Horay!
    return total


# Graph Algorithm
def count_bags_graph(graph, start, multiplier = 1):
    
    # Count the number of bags along the way
    total = 0
    
    # If terminal node, terminate immediately
    if graph[start]:
        
        # Target nodes
        for node, weight in graph[start].items():
            
            # Collect the total and create temp multiplier
            x = multiplier * weight
            total += x
            total += count_bags_graph(graph, node, x)
            
    return total

220149


220149

### Measure performance of each method (Puzzle Data)

In [ ]:
# Let's compare speeds for the sample question

# Matrix
start_matrix = time.time()
answer_matrix = count_bags_matrix(matrix_3, keys["shiny gold"]) # Answer
end_matrix = time.time()

# Graph
start_graph = time.time()
answer_graph = count_(graph_3, "shiny gold")) # Answer
end_graph = time.time()

# Result
print("Matrix method answer is {}, it took {:.2}s to process this request".format(answer_matrix, end_matrix - start_matrix))
print("Graph method answer is {}, it took {:.2}s to process this request".format(answer_graph, end_graph - start_graph))